# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Problema 3
#### Aluno: Eduardo Cesar Cauduro Coelho
#### Matrícula: 202310175
#### Aluno: Felipe Geraldo de Oliveira
#### Matrícula: 202310174

## Instalação da biblioteca Gurobipy

In [1]:
!pip install gurobipy

## Para executar, siga os seguintes passos:
* Extraia o arquivo "Dados McDonalds AP2.zip"
* Crie um diretório chamado McDonalds no diretório atual do Google Colab
* Faça upload de todos os arquivos wsv extraídos para essa pasta McDonalds

## Importação da biblioteca re para expressões regulares e leitura dos dados do arquivo McDonalds-amnt.wsv

#### A expressão regular "([^"]*)"|(\S+) apresenta dois grupos de captura:

* "([^"]*)" captura tudo que está entre aspas, por exemplo: "Big Mac".
* (\S+) captura tudo que não é espaço, por exemplo: Energia.

#### Se a palavra está entre aspas, o primeiro grupo captura tudo até que encontre o fecho das aspas.
#### Se a palavra não está entre aspas, o segundo grupo captura tudo até encontrar um espaço.

In [2]:
import re

amnt = {}
composition_nutr = []
products = []

counter = 0

with open('McDonalds/McDonalds-amnt.wsv', 'r') as file:
    for line in file:

        parts = re.findall(r'"([^"]*)"|(\S+)', line.strip())

        if counter == 0: # Ignora o cabeçalho
          counter += 1
          continue

        row = [part[0] if part[0] else part[1] for part in parts] # Quando um grupo é capturado, o outro é nulo. Escolhe o que não é nulo.

        if counter == 1: # Captura os nomes dos nutrientes
          composition_nutr = row[1:]
          counter += 1
          continue

        nutrients = {c: n for c, n in zip(composition_nutr, row[1:])}

        products.append(row[0])

        amnt[row[0]] = nutrients
        counter +=1

print(amnt["Big Mac"]) # Impressão de uma instância

{'Energia': '25', 'Carb.': '14', 'Prot.': '33', 'G. Total': '49', 'G. Sat.': '55', 'Colest.': '18', 'Fibra': '14', 'Sódio': '40', 'Cálcio': '19', 'Ferro': '46'}


## Leitura dos dados do arquivo McDonalds-food.wsv

#### Como os limites inferiores e superiores são todos 'NA', então não considera esses dados na leitura:

In [3]:
food = {}

data_index = ["cost", "veg"]

counter = 0

with open('McDonalds/McDonalds-food.wsv', 'r') as file:
    for line in file:

        parts = re.findall(r'"([^"]*)"|(\S+)', line.strip())

        if counter < 3: # Ignora o cabeçalho e os nomes das colunas
          counter += 1
          continue

        row = [part[0] if part[0] else part[1] for part in parts if part[0] or part[1] != "NA"] # Faz o mesmo que na leitura do arquivo anterior, mas ignora os dados que são NA

        data = {c: n for c, n in zip(data_index, row[1:])}

        food[row[0]] = data

        counter +=1

print(food["Big Mac"]) # Impressão de uma instância

{'cost': '6.90', 'veg': 'false'}


## Como o arquivo McDonalds-nutr.wsv possui n_min = 100 e n_max = 'NF' (NF nesse caso seria equivalente a ilimitado) para todo nutriente, então não é necessário ler todo o arquivo, basta anotar o valor de n_min.

In [4]:
VDR_MIN = 100

## Criação do modelo:

In [5]:
from gurobipy import Model, GRB, quicksum

model = Model("foods_McDonalds")

Restricted license - for non-production use only - expires 2026-11-23


## Declaração das variáveis de decisão que indicam o número de alimentos de cada tipo comprados:

In [6]:
var_product = {}
for product in products:
  var_product[product] = model.addVar(vtype=GRB.CONTINUOUS, name=product, lb=0)

## Função objetivo: minimiza o custo de compra dos alimentos:

In [7]:
obj_sum = quicksum(var_product[product] * food[product]["cost"] for product in products)

model.setObjective(obj_sum, GRB.MINIMIZE)

## Restrição

* A oferta nutricional dos alimentos comprados deve ser maior ou igual ao valor diário mínimo:


In [8]:
for nutrient in composition_nutr:

  nutr_constr = quicksum(var_product[product] * amnt[product][nutrient] for product in products)

  model.addConstr(nutr_constr >= VDR_MIN, name=f'nutriente_{nutrient}')

## Portanto, temos o seguinte modelo:

\begin{equation}
\min f(x_{ij}) = \sum_{i=1}^{n} (c_i \times x_i)
\end{equation}

sujeito a:

\begin{alignat}{2}
\sum_{i=1}^{n}(x_{i} \times n_{ij}) \ge V && \qquad j = 1,...,m \\
x_{i} \ge 0 && \qquad
\end{alignat}

* Sendo:
  * $x_i$ é a variável que representa a quantidade do alimento i comprado, onde i = $1,...,n$.
  * $c_i$ é o parâmetro que representa o custo do produto i.
  * $n_{ij}$ é o parâmetro que representa o percentual diário que um alimento i fornece de um nutriente j, onde j = $1,...,m$
  * V representa o valor diário mínimo necessário de cada nutriente.

# Otimização do modelo:

In [9]:
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 84 columns and 634 nonzeros
Model fingerprint: 0x8971e040
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 0 rows and 16 columns
Presolve time: 0.02s
Presolved: 10 rows, 68 columns, 580 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.500000e+01   0.000000e+00      0s
       3    2.4311927e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.03 seconds (0.00 work units)
Optimal objective  2.431192661e+01


## Impressão das soluções do modelo:

In [10]:
print(f"Valor ótimo: {model.objVal}")
for product in products:
  print(f"{product}: {var_product[product].x}")

Valor ótimo: 24.31192660550459
Big Mac: 0.0
Big Tasty: 0.0
Quarterão com Queijo: 0.0
McNífico Bacon: 0.0
Cheddar McMelt: 0.0
McFish: 0.0
McChicken: 0.0
Crispy Chicken: 0.0
Chicken Grill: 0.0
McDuplo: 5.045871559633028
Cheeseburger: 0.0
Hamburger: 0.0
Queijo Quente: 0.0
McBacon Junior: 0.0
Mini McSalad Shaker: 0.0
Premium Salad: 0.0
Premium Salad Grill: 0.0
Premium Salad Crispy: 0.0
Molho Salada Caseiro: 0.0
Molho Salada Tomate Seco: 0.0
Molho Salada Ranch: 0.0
McFritas grande: 0.0
McFritas média: 0.0
McFritas pequena: 0.0
Chicken McNuggets (12 unid.): 0.0
Chicken McNuggets (4 unid.): 0.0
Chicken McNuggets (6 unid.): 0.0
Molho Agridoce: 0.0
Molho Barbecue: 0.0
Molho Hot Mustard: 0.0
Molho Caipira: 0.0
McFruit Uva (300 ml): 0.0
McFruit Uva (500 ml): 0.0
McFruit Maracujá (300 ml): 0.0
McFruit Maracujá (500 ml): 0.0
McFruit Limão (300 ml): 0.0
McFruit Limão (500 ml): 0.0
McFruit Laranja (300 ml): 0.0
McFruit Laranja (500 ml): 0.0
Água de Coco: 0.0
Água de Coco do Ronald: 0.0
Água Mineral: 

# 2) Análise de sensibilidade:

In [11]:
print("\nIntervalos de Custos (Vetor de Custos):")
for variavel in model.getVars():
  print(f"Variável {variavel.VarName}:")
  print(f"  Lower bound do custo (SAObjLow): {variavel.SAObjLow}")
  print(f"  Upper bound do custo (SAObjUp): {variavel.SAObjUp}")

print("\nIntervalos de Recursos (Vetor de Recursos):")
for restricao in model.getConstrs():
  print(f"Restrição {restricao.ConstrName}:")
  print(f"  Lower bound do recurso (SARHSLow): {restricao.SARHSLow}")
  print(f"  Upper bound do recurso (SARHSUp): {restricao.SARHSUp}")
  print(f"  Variavel dual: {restricao.pi}")
  print(f"  Folga primal: {restricao.slack}")


Intervalos de Custos (Vetor de Custos):
Variável Big Mac:
  Lower bound do custo (SAObjLow): 3.8954128440366977
  Upper bound do custo (SAObjUp): inf
Variável Big Tasty:
  Lower bound do custo (SAObjLow): 6.279816513761467
  Upper bound do custo (SAObjUp): inf
Variável Quarterão com Queijo:
  Lower bound do custo (SAObjLow): 5.077064220183487
  Upper bound do custo (SAObjUp): inf
Variável McNífico Bacon:
  Lower bound do custo (SAObjLow): 5.4779816513761475
  Upper bound do custo (SAObjUp): inf
Variável Cheddar McMelt:
  Lower bound do custo (SAObjLow): 4.676146788990826
  Upper bound do custo (SAObjUp): inf
Variável McFish:
  Lower bound do custo (SAObjLow): 3.144036697247707
  Upper bound do custo (SAObjUp): inf
Variável McChicken:
  Lower bound do custo (SAObjLow): 3.441284403669725
  Upper bound do custo (SAObjUp): inf
Variável Crispy Chicken:
  Lower bound do custo (SAObjLow): 3.7724770642201837
  Upper bound do custo (SAObjUp): inf
Variável Chicken Grill:
  Lower bound do custo 

##Custos:
* Todos os produtos que não estão na base têm Upper Bound ∞, ou seja, já não valem a pena e continuarão assim se os seus custos aumentarem.

* Quanto menos nutrientes um alimento fornece, menor deve ser o seu preço para que valha a pena entrar na base. Portanto, o Lower Bound destes alimentos é menor.

* Alimentos que têm nutrientes próximos de zero, como água mineral e tipos de Coca-Cola, têm 0 de Lower Bound, pois não são eficazes para atingir a meta diária de nutrientes mesmo que os seus preços forem R\$0,00.

* Maçã, casquinha de baunilha e McDuplo fazem parte da base, mas sairão dela de os seus preços passarem de R\$2,30, R\$1,50 e R\$4,24, respectivamente. Por outro lado, estes alimentos terão um custo benefício ainda maior e também mudarão a base selecionado se os seus preços forem menores que R\$0,57, R\$0,76 e R\$2,80. (todos os valores foram aproximados para 2 casas decimais)

##Recursos:
* Nutrientes com Lower Bound -∞ indicam que a base não será alterada mesmo que estes não sejam necessários, pois os alimentos escolhidos são os quais possuem o melhor custo-benefício para o restante deles.

* Colesterol e Fibra possuem Lower Bounds iguais a 60,18465089440275 e 54,54545454545455, respectivamente. Isto significa que, caso as quantidades necessárias destes nutrientes forem menores do que estes valores, a base será alterada, pois eles terão menos influência no resultado ótimo e outros alimentos que têm um custo-benefício maior para outros nutrientes entrarão na base.

* Todos os nutrientes possuem um Upper Bound finito, então existe um conjunto de alimentos com custo-benefício maior para um aumento suficiente de qualquer nutriente.

* Apenas as variáveis duais de Colesterol e Fibra não são nulas, ou seja, somente a variação destes nutrientes pode alterar o valor da função objetivo. O valor da variável dual do Colesterol é maior, então a variação dele tem mais influência sobre o valor da função objetivo.

#3) Modelo para dieta ótima vegetariana:

In [12]:
model_vegetarian = Model("foods_McDonalds_vegetarian")

## Declaração das variáveis de decisão que indicam o número de alimentos vegetarianos de cada tipo comprados:

In [13]:
var_product_vegetarian = {}

vegetarian_products = []
for product in products: # Seleciona os produtos veganos dentre todos os produtos
  if(food[product]["veg"] == "true"):
    vegetarian_products.append(product)

for product in vegetarian_products:
    var_product_vegetarian[product] = model_vegetarian.addVar(vtype=GRB.CONTINUOUS, name=product, lb=0)

## Função objetivo: minimiza o custo de compra dos alimentos:

In [14]:
obj_sum = quicksum(var_product_vegetarian[product] * food[product]["cost"] for product in vegetarian_products)

model_vegetarian.setObjective(obj_sum, GRB.MINIMIZE)

## Restrição

* A oferta nutricional dos alimentos comprados deve ser maior ou igual ao valor diário mínimo:


In [15]:
for nutrient in composition_nutr:

  nutr_constr_veg = quicksum(var_product_vegetarian[product] * amnt[product][nutrient] for product in vegetarian_products)

  model_vegetarian.addConstr(nutr_constr_veg >= VDR_MIN, name=f'nutriente_{nutrient}')

## Como mostrado nas restrições, o modelo é idêntico ao modelo anterior, com a diferença que este modelo considera apenas os alimentos vegetarianos, excluindo os que não são vegetarianos.

## Otimização do modelo:

In [16]:
model_vegetarian.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 67 columns and 464 nonzeros
Model fingerprint: 0xdda3ee70
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 0 rows and 15 columns
Presolve time: 0.01s
Presolved: 10 rows, 52 columns, 415 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.750000e+01   0.000000e+00      0s
       4    3.9634146e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.963414634e+01


## Impressão das soluções do modelo:

In [17]:
print(f"Valor ótimo: {model_vegetarian.objVal}")
for product in vegetarian_products:
  print(f"{product}: {var_product_vegetarian[product].x}")

Valor ótimo: 39.63414634146341
Queijo Quente: 0.0
Mini McSalad Shaker: 0.0
Premium Salad: 0.0
Premium Salad Crispy: 0.0
Molho Salada Caseiro: 0.0
Molho Salada Tomate Seco: 0.0
Molho Salada Ranch: 7.317073170731705
McFritas grande: 0.0
McFritas média: 0.0
McFritas pequena: 0.0
Molho Agridoce: 0.0
Molho Barbecue: 0.0
Molho Hot Mustard: 0.0
Molho Caipira: 0.0
McFruit Uva (300 ml): 0.0
McFruit Uva (500 ml): 0.0
McFruit Maracujá (300 ml): 0.0
McFruit Maracujá (500 ml): 0.0
McFruit Limão (300 ml): 0.0
McFruit Limão (500 ml): 0.0
McFruit Laranja (300 ml): 0.0
McFruit Laranja (500 ml): 0.0
Água de Coco: 0.0
Água de Coco do Ronald: 0.0
Água Mineral: 0.0
Água Mineral com gás: 0.0
Coca-Cola (300 ml): 0.0
Coca-Cola (500 ml): 0.0
Coca-Cola (700 ml): 0.0
Fanta (300 ml): 0.0
Fanta (500 ml): 0.0
Fanta (700 ml): 0.0
Coca-Cola Light (300 ml): 0.0
Coca-Cola Light (500 ml): 0.0
Coca-Cola Light (700 ml): 0.0
Guaraná (300 ml): 0.0
Guaraná (500 ml): 0.0
Guaraná (700 ml): 0.0
Guaraná Zero Açúcar (300 ml): 0.0

#4) Modelo para dieta ótima de alimentos únicos:

In [18]:
model_unique = Model("foods_McDonalds_unique")

## Este modelo é muito semelhante ao primeiro modelo, sendo a única diferença é que as variáveis neste modelo são configuradas como binárias, pois o alimento só pode ser consumido uma única vez, ou seja, ou ele é consumido (1) ou não é (0). Todo o resto se mantém.

In [19]:
var_product_unique = {}
for product in products:
  var_product_unique[product] = model_unique.addVar(vtype=GRB.BINARY, name=product, lb=0) # Variáveis binárias

In [20]:
obj_sum = quicksum(var_product_unique[product] * food[product]["cost"] for product in products)

model_unique.setObjective(obj_sum, GRB.MINIMIZE)

In [21]:
for nutrient in composition_nutr:

  nutr_constr = quicksum(var_product_unique[product] * amnt[product][nutrient] for product in products)

  model_unique.addConstr(nutr_constr >= VDR_MIN, name=f'nutriente_{nutrient}')

## Otimização do modelo:

In [22]:
model_unique.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 84 columns and 634 nonzeros
Model fingerprint: 0x27ae825d
Variable types: 0 continuous, 84 integer (84 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective 94.1500000
Presolve removed 1 rows and 5 columns
Presolve time: 0.00s
Presolved: 9 rows, 79 columns, 561 nonzeros
Variable types: 0 continuous, 79 integer (79 binary)

Root relaxation: objective 3.156278e+01, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   31.562

## Impressão das soluções do modelo:

In [23]:
print(f"Valor ótimo: {model_unique.objVal}")
for product in products:
  print(f"{product}: {var_product_unique[product].x}")

Valor ótimo: 32.55
Big Mac: -0.0
Big Tasty: 0.0
Quarterão com Queijo: 1.0
McNífico Bacon: 1.0
Cheddar McMelt: -0.0
McFish: -0.0
McChicken: -0.0
Crispy Chicken: -0.0
Chicken Grill: -0.0
McDuplo: 1.0
Cheeseburger: 1.0
Hamburger: 1.0
Queijo Quente: 0.0
McBacon Junior: 0.0
Mini McSalad Shaker: -0.0
Premium Salad: -0.0
Premium Salad Grill: -0.0
Premium Salad Crispy: -0.0
Molho Salada Caseiro: -0.0
Molho Salada Tomate Seco: -0.0
Molho Salada Ranch: -0.0
McFritas grande: 1.0
McFritas média: 0.0
McFritas pequena: 0.0
Chicken McNuggets (12 unid.): -0.0
Chicken McNuggets (4 unid.): -0.0
Chicken McNuggets (6 unid.): -0.0
Molho Agridoce: -0.0
Molho Barbecue: -0.0
Molho Hot Mustard: -0.0
Molho Caipira: -0.0
McFruit Uva (300 ml): -0.0
McFruit Uva (500 ml): -0.0
McFruit Maracujá (300 ml): -0.0
McFruit Maracujá (500 ml): -0.0
McFruit Limão (300 ml): -0.0
McFruit Limão (500 ml): -0.0
McFruit Laranja (300 ml): -0.0
McFruit Laranja (500 ml): -0.0
Água de Coco: -0.0
Água de Coco do Ronald: -0.0
Água Miner